In [4]:
# Importación de librerías que se utilizarán
import os
import re
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc, confusion_matrix, precision_score, recall_score, f1_score

import plotly.express as px # Crear gráficos interactivos
import matplotlib.pyplot as plt
from matplotlib import style # Cambiar el estilo visual de los gráficos
import seaborn as sns

import mlflow
import mlflow.sklearn

import nltk #  Librería para procesamiento de lenguaje natural
from nltk.util import ngrams # Crear n-gramas

import pycaret
from pycaret.classification import * # Módulo de PyCaret para problemas de clasificación
from pycaret.utils import version # Obtener la versión de PyCaret instalada
#from pycaret.nlp import * # Módulo de PyCaret para procesamiento de texto

import spacy # Para procesamiento de lenguaje natural.
from spacy.lang.en.examples import sentences # Da ejemplos de frases en español para pruebas.
from spacy.lang.en.stop_words import STOP_WORDS # Stop words en inglés
from nltk.corpus import stopwords # Lista de stopwords pero de la librería NLTK (más general).

import pyLDAvis # Para visualizar modelos de temas
#import pyLDAvis.gensim_models
pyLDAvis.enable_notebook() # Permite que las visualizaciones de pyLDAvis se vean directamente en el notebook
version() # muestra la versión de PyCaret

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    # Este bloque desactiva los mensajes de advertencia para no "ensuciar" la salida de resultados en la consola o notebook.


Hubo que instalar lo siguiente: 
* !pip install pycaret
* !python -m spacy download en_core_web_sm
* !python -m spacy download es_core_news_sm
* !pip install pyLDAvis==3.2.2

## 1. Web scraping
This endpoint will receive as input Web scrapping:
- Open web page
- Read the content
- Extract content
- Save all in a data frame

Por el momento el web scraping se realizará con un archivo previamente desacargado de [Kaggle](https://www.kaggle.com/datasets/davidgauthier/glassdoor-job-reviews), para lo que lo cargaremos 

In [5]:
# Cargamos el archivo con el que trabajaremos
file_path = "C:/Users/palom/OneDrive/Dokumen/CUCEA/2DO_SEM/Challenges-Progra2/Challenge_2/data/glassdoor_reviews.csv"
df = pd.read_csv(file_path)
# Mostramos las primeras filas del archivo para analizar su estructura
df.head()

,firm,date_review,job_title,current,location,overall_rating,work_life_balance,culture_values,diversity_inclusion,career_opp,comp_benefits,senior_mgmt,recommend,ceo_approv,outlook,headline,pros,cons
0,AFH-Wealth-Management,2015-04-05,,Current Employee,NaN,2,4.0,3.0,NaN,2.0,3.0,3.0,x,o,r,"Young colleagues, poor micro management",Very friendly and welcoming to new staff. Easy...,"Poor salaries, poor training and communication."
1,AFH-Wealth-Management,2015-12-11,Office Administrator,"Current Employee, more than 1 year","Bromsgrove, England, England",2,3.0,1.0,NaN,2.0,1.0,4.0,x,o,r,"Excellent staff, poor salary","Friendly, helpful and hard-working colleagues",Poor salary which doesn't improve much with pr...
2,AFH-Wealth-Management,2016-01-28,Office Administrator,"Current Employee, less than 1 year","Bromsgrove, England, England",1,1.0,1.0,NaN,1.0,1.0,1.0,x,o,x,"Low salary, bad micromanagement",Easy to get the job even without experience in...,"Very low salary, poor working conditions, very..."
3,AFH-Wealth-Management,2016-04-16,,Current Employee,NaN,5,2.0,3.0,NaN,2.0,2.0,3.0,x,o,r,Over promised under delivered,Nice staff to work with,No career progression and salary is poor
4,AFH-Wealth-Management,2016-04-23,Office Administrator,"Current Employee, more than 1 year","Bromsgrove, England, England",1,2.0,1.0,NaN,2.0,1.0,1.0,x,o,x,client reporting admin,"Easy to get the job, Nice colleagues.","Abysmal pay, around minimum wage. No actual tr..."


In [6]:
print(f"Dimensiones del dataframe: {df.shape}")
# obtenemos información del tipo de dato por columna
df.info()

Dimensiones del dataframe: (838566, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838566 entries, 0 to 838565
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   firm                 838566 non-null  object 
 1   date_review          838566 non-null  object 
 2   job_title            838566 non-null  object 
 3   current              838566 non-null  object 
 4   location             541223 non-null  object 
 5   overall_rating       838566 non-null  int64  
 6   work_life_balance    688672 non-null  float64
 7   culture_values       647193 non-null  float64
 8   diversity_inclusion  136066 non-null  float64
 9   career_opp           691065 non-null  float64
 10  comp_benefits        688484 non-null  float64
 11  senior_mgmt          682690 non-null  float64
 12  recommend            838566 non-null  object 
 13  ceo_approv           838566 non-null  object 
 14  outlook              838566 

Se observa que de las 18 variables, 11 son de tipo object, de las cuales 4 de ellas contienen registros vacíos (location, headline, pros, cons).
Observado esto se creará un nuevo dataframe con solo variables _object_ y será este al que se le aplicará el NLP.

In [7]:
df_text = df.select_dtypes(include='object')

# Imprimimos los primeros registros del dataframe para visualizar el filtrado
df_text.head()

,firm,date_review,job_title,current,location,recommend,ceo_approv,outlook,headline,pros,cons
0,AFH-Wealth-Management,2015-04-05,,Current Employee,NaN,x,o,r,"Young colleagues, poor micro management",Very friendly and welcoming to new staff. Easy...,"Poor salaries, poor training and communication."
1,AFH-Wealth-Management,2015-12-11,Office Administrator,"Current Employee, more than 1 year","Bromsgrove, England, England",x,o,r,"Excellent staff, poor salary","Friendly, helpful and hard-working colleagues",Poor salary which doesn't improve much with pr...
2,AFH-Wealth-Management,2016-01-28,Office Administrator,"Current Employee, less than 1 year","Bromsgrove, England, England",x,o,x,"Low salary, bad micromanagement",Easy to get the job even without experience in...,"Very low salary, poor working conditions, very..."
3,AFH-Wealth-Management,2016-04-16,,Current Employee,NaN,x,o,r,Over promised under delivered,Nice staff to work with,No career progression and salary is poor
4,AFH-Wealth-Management,2016-04-23,Office Administrator,"Current Employee, more than 1 year","Bromsgrove, England, England",x,o,x,client reporting admin,"Easy to get the job, Nice colleagues.","Abysmal pay, around minimum wage. No actual tr..."


## 2. Model to classifier and make a sentiment analysis
This part will contein 3 sections:
* Text preprocessing
* Classification proposed
* Extraction of main features

In [9]:
nlp = spacy.load("en_core_web_sm")
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\palom\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [20]:
#### TEXT PREPROCESSING
def limpiar(texto):
    '''
    Esta función limpia y tokeniza el texto en palabras individuales.
    El orden en el que se va limpiando el texto no es arbitrario.
    El listado de signos de puntuación se ha obtenido de: print(string.punctuation)
    y re.escape(string.punctuation)
    '''
    # Se convierte todo el texto en str
    nuevo_texto = str(texto)
    # Se convierte todo el texto a minúsculas
    nuevo_texto = nuevo_texto.lower()
    # Eliminación de páginas web (palabras que empiezan por "http")
    nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
    # Eliminación de signos de puntuación
    regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto)
    # Eliminación de números
    nuevo_texto = re.sub("\d+", ' ', nuevo_texto)
    # Eliminación de espacios en blanco múltiples
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    
    return(nuevo_texto)

In [ ]:
# Se aplica la función limpiar a la columna -pros-
# ==============================================================================
df_text['pros'] = df_text['pros'].apply(lambda col: limpiar(col))
df_text

,firm,date_review,job_title,current,location,recommend,ceo_approv,outlook,headline,pros,cons
0,AFH-Wealth-Management,2015-04-05,,Current Employee,NaN,x,o,r,"Young colleagues, poor micro management",very friendly and welcoming to new staff easy ...,"Poor salaries, poor training and communication."
1,AFH-Wealth-Management,2015-12-11,Office Administrator,"Current Employee, more than 1 year","Bromsgrove, England, England",x,o,r,"Excellent staff, poor salary",friendly helpful and hard working colleagues,Poor salary which doesn't improve much with pr...
2,AFH-Wealth-Management,2016-01-28,Office Administrator,"Current Employee, less than 1 year","Bromsgrove, England, England",x,o,x,"Low salary, bad micromanagement",easy to get the job even without experience in...,"Very low salary, poor working conditions, very..."
3,AFH-Wealth-Management,2016-04-16,,Current Employee,NaN,x,o,r,Over promised under delivered,nice staff to work with,No career progression and salary is poor
4,AFH-Wealth-Management,2016-04-23,Office Administrator,"Current Employee, more than 1 year","Bromsgrove, England, England",x,o,x,client reporting admin,easy to get the job nice colleagues,"Abysmal pay, around minimum wage. No actual tr..."
...,...,...,...,...,...,...,...,...,...,...,...
838561,the-LEGO-Group,2021-06-02,Marketing Manager,"Current Employee, more than 5 years","München, Bavaria, Bavaria",v,v,v,Just an awesome company to work for!!!,great company values awesome product smart col...,Not very easy to transfer to other locations
838562,the-LEGO-Group,2021-06-03,Sales Associate,"Current Employee, less than 1 year","London, England, England",o,o,o,working at lego,staff discount is really nice,micro managing is a hassle\r\ncan become menta...
838563,the-LEGO-Group,2021-06-03,Strategist,Current Employee,NaN,v,o,o,not interested in growing their people,loved brand for a lot of people,you can spend 6-10 years without any promotion...
838564,the-LEGO-Group,2021-06-04,Customer Service Representative,"Current Employee, less than 1 year",NaN,o,o,o,Great Place to Work,good wages good hours lots of resources,"Working every other weekend, busy seasons can ..."


## 3. Create a pipline to MLOps